Video Link: https://www.youtube.com/watch?v=06VErVj9MaQ&t=1108s

Numba derives from "Numpy" and "Mamba". Numba turns Python into a compiled language with a GPU target.

[You cannot use the python list and dictionary](https://numba.pydata.org/numba-doc/dev/cuda/cudapysupported.html). If you write in that way, it might be slower. But you can use Numpy array.
```
The following Python constructs are not supported:
- Exception handling
- context management (the with statement)
- Comprehensions (either list, dict, set or generator comprehensions)
- Generator (any yield statments)
```

# Numba

- Opernsource BSD license
- Basic CUDA GPU JIT compilation
- OpenCL support coming

In [1]:
import numba
print("numba", numba.__version__)

numba 0.34.0


# The CUDA GPU

- A massively parallel processor (many cores)
    - 100 threads, 1000 threads, and more
- optimized for data throughput
    - simple (shallow) cache hierarchy
    - best with manual caching!
    - Cache memory is called shared memory and it is addressable
- CPU is latency optimized
    - Deep cache hierarchy
    - L1, L2 L3 cahces
- GPU execution model is different
- GPU forces you to think and program in parallel

In [3]:
import numba.cuda
import numpy as np
import math

In [4]:
my_gpu = numba.cuda.get_current_device()
print("Running on GPU:", my_gpu.name)

Running on GPU: b'GeForce GTX 1080 Ti'


In [6]:
cc = my_gpu.compute_capability
print("Compute capability: ", "%d.%d" % cc, "(Numba requires >= 2.0)")

Compute capability:  6.1 (Numba requires >= 2.0)


In [8]:
print("Number of streaming multiprocessor:", my_gpu.MULTIPROCESSOR_COUNT)

Number of streaming multiprocessor: 28


# High-level Array-Oriented Style
- Use NumPy array as a unit of computation
- Use NumPy universal function (ufunc) as an abstraction of computation of scheduling
- ufuncs are elementwise functions
- If you use NumPy, you are using ufuncs

In [9]:
print(np.sin, "is of type", type(np.sin))
print(np.add, "is of type", type(np.add))

<ufunc 'sin'> is of type <class 'numpy.ufunc'>
<ufunc 'add'> is of type <class 'numpy.ufunc'>


# Vectorize
- generate a ufunc from a python function
- converts scalar function to elementwise array function
- Numba provides CPU support
-  <s>NumbaPro provides GPU support</s>
    - https://docs.anaconda.com/numbapro/

In [36]:
# CPU version
@numba.vectorize(['float32(float32, float32)', 
                  'float64(float64, float64)'], target = 'cpu')
def cpu_sincos(x, y):
    return math.sin(x) * math.cos(y)

# CPU version (multicore)
@numba.vectorize(['float32(float32, float32)', 
                  'float64(float64, float64)'], target = 'parallel')
def cpu_sincos_mc(x, y):
    return math.sin(x) * math.cos(y)

Reference: 
- https://numba.pydata.org/numba-doc/latest/cuda/ufunc.html
- https://numba.pydata.org/numba-doc/dev/user/vectorize.html

```
The vectorize() decorator supports multiple ufunc targets:
Target      Description
cpu         Single-threaded CPU
parallel    Multi-core CPU
cuda        CUDA GPU
```

In [33]:
# GPU version
@numba.vectorize(['float32(float32, float32)', 
                  'float64(float64, float64)'], target = 'cuda')
def gpu_sincos(x, y):
    return math.sin(x) * math.cos(y)

Note: Types and signatures in Numba
- https://numba.pydata.org/numba-doc/dev/reference/types.html
```
An example function signature would be the string "f8(i4, i4)" (or the equivalent "float64(int32, int32)") which specifies a function taking two 32-bit integers and returning a double-precision float.
```

# Test it out
- 2 input arrays
- 1 output array
- 1 million doubles (8 MB) per array
- Total 24 MB of data

Note: [numpy.allclose](https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.allclose.html)  
Returns True if two arrays are element-wise equal within a tolerance.

In [38]:
# generate data
n = 1000000
x = np.linspace(0, np.pi, n)
y = np.linspace(0, np.pi, n)

# check result
np_ans = np.sin(x) * np.cos(y)
np_cpu_ans = cpu_sincos(x, y)
np_cpumc_ans = cpu_sincos_mc(x, y)
np_gpu_ans = gpu_sincos(x, y)


print("CPU vectorize correct: ", np.allclose(np_cpu_ans, np_ans))
print("CPU vectorize correct: ", np.allclose(np_cpumc_ans, np_ans))
print("GPU vectorize correct: ", np.allclose(np_gpu_ans, np_ans))

CPU vectorize correct:  True
CPU vectorize correct:  True
GPU vectorize correct:  True


## Benchmark: calculate the time of each method.
**Results:** 
- CPU vectorize time is similar to pure Numpy time because sin() and cos() calls dominate the time
- CPU vectorize is a lot faster

In [40]:
print("Numpy")
%timeit np.sin(x) * np.cos(y)

print("CPU vectorize")
%timeit cpu_sincos(x, y)

print("CPU vectorize (multicore)")
%timeit cpu_sincos_mc(x, y)

print("GPU vectorize")
%timeit gpu_sincos(x, y)

Numpy
28.4 ms ± 11.8 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
CPU vectorize
31.5 ms ± 35.9 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
CPU vectorize (multicore)
3.97 ms ± 63.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
GPU vectorize
6.32 ms ± 168 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Note: The data must be transfer from CPU ram to the GPU global memory. The results is written back from the GPU global memory to CPU ram

In [41]:
# delete the objects
del x, y

# Behind teh scene

### Automatic memory transfer
- Numpy arrays are automatically transferred
    - CPU -> GPU
    - GPU -> CPU
    
### Automatic work scheduling
- The work is distributed the across all threads on the GPU
- The GPU hardware handles the scheduling

### Automatic GPU memory management
- GPU memory is tied to object lifetime
- freed automatically

# Another Vectorize Example

In [27]:
@numba.vectorize(
    ['float32(float32, float32, float32, float32)'],
    target = "cpu")
def cpu_powers(p, q, r, s):
    return math.sqrt(p**2 + q**3 + r**4 + s**5)

In [28]:
@numba.vectorize(
    ['float32(float32, float32, float32, float32)'],
    target = "cuda")
def gpu_powers(p, q, r, s):
    return math.sqrt(p**2 + q**3 + r**4 + s**5)

In [29]:
# generate data
n = 5000000
p = np.random.random(n).astype(np.float32)
q = np.random.random(n).astype(np.float32)
r = np.random.random(n).astype(np.float32)
s = np.random.random(n).astype(np.float32)

# Check results
np_ans = np.sqrt(p**2 + q**3 + r**4 + s**5)
cpu_ans = cpu_powers(p, q, r, s)
gpu_ans = gpu_powers(p, q, r, s)

print("CPU vectorize correct: ", np.allclose(cpu_ans, np_ans))
print("GPU vectorize correct: ", np.allclose(gpu_ans, np_ans))

CPU vectorize correct:  True
GPU vectorize correct:  True


## Benchmark

In [30]:
print("Numpy")
%timeit np.sqrt(p**2 + q**3 + r**4 + s**5)

print("CPU vectorize")
%timeit cpu_powers(p, q, r, s)

print("GPU vectorize")
%timeit gpu_powers(p, q, r, s)

Numpy
679 ms ± 5.85 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
CPU vectorize
9.82 ms ± 16.2 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
GPU vectorize
21.7 ms ± 422 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


I wonder why in my examples, the GPU vectorize is slower than the CPU vectorize

In [31]:
del p, q, r, s

# Generalized Universal Function (guvectorize)
- vectorize is limited to scalar arguments in the core function
- GUVectorize accepts array arguments

reference
- https://numba.pydata.org/numba-doc/latest/cuda/ufunc.html
- https://numba.pydata.org/numba-doc/dev/user/vectorize.html


"While **vectorize()** allows you to write ufuncs that work on one element at a time, the guvectorize() decorator takes the concept one step further and allows you to write ufuncs that will work on an arbitrary number of elements of input arrays, and take and return arrays of differing dimensions. The typical example is a running median or a convolution filter.

Contrary to vectorize() functions, guvectorize() functions don’t return their result value: they take it as an array argument, which must be filled in by the function. This is because the array is actually allocated by NumPy’s dispatch mechanism, which calls into the Numba-generated code."


In [44]:
@numba.guvectorize(
    ['void(float32[:, :], float32[:, :], float32[:, :])'],
    '(m, n), (n, p) -> (n, p)', 
    target = "cuda")
def batch_matrix_mult(a, b, c):
    for i in range(c.shape[0]):
        for j in range(c.shape[1]):
            tmp = 0
            for n in range(a.shape[1]):
                tmp += a[i, n] * b[n, j]
            c[i, j] = tmp

### Test the function

In [47]:
a = np.arange(1.0, 10.0, dtype = np.float32).reshape(3, 3)
b = np.arange(1.0, 10.0, dtype = np.float32).reshape(3, 3)

# use the builtin matrix multiply in numpy for CPU test
import numpy.core.umath_tests as ut

# Check result
print("NumPy result")
np_ans = ut.matrix_multiply(a, b)
print(np_ans)

print("NumPy GPU result")
gpu_ans = batch_matrix_mult(a, b)
print(gpu_ans)

assert np.allclose(np_ans, gpu_ans)

NumPy result
[[ 30.  36.  42.]
 [ 66.  81.  96.]
 [102. 126. 150.]]
NumPy GPU result
[[ 30.  36.  42.]
 [ 66.  81.  96.]
 [102. 126. 150.]]


### Benchmark
Test it out
- Batch multiply two 4 million 2x2 matrices

In [51]:
n = 4000000
dim = 2
a = np.random.random(n * dim * dim). \
    astype(np.float32). \
    reshape(n, dim, dim)

b = np.random.random(n * dim * dim). \
    astype(np.float32). \
    reshape(n, dim, dim)

print("Numpy time")
%timeit ut.matrix_multiply(a, b)

print("Numpy GPU time")
%timeit batch_matrix_mult(a, b)

Numpy time
77.6 ms ± 171 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Numpy GPU time
120 ms ± 885 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


- GPU time seems to be similar to CPU time (in my example, the GPU time is even higher than CPU time)
- It is because the memory transfer
    - 

# Manually Transer the data to the GPU

- This will let us see the actual compute time without the CPU <-> GPU transfer
- **numba.cuda.device_array_like** allocate without initialization with the type and shape of another array.
    - similar to numpy.empty_like(a)
- **numba.cuda.to_device** create a GPU copy of the CPU array

In [52]:
# Note this does not copy a to GPU
dc = numba.cuda.device_array_like(a)

# copy
da = numba.cuda.to_device(a)
db = numba.cuda.to_device(b)

In [53]:
def check_pure_compute_time(da, db, dc):
    batch_matrix_mult(da, db, out = dc)
    numba.cuda.synchronize() # ensure the call has completed
    
%timeit check_pure_compute_time(da, db, dc)

999 µs ± 1.28 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


- actual compute time is a lot faster
- PCI-express transfer overhead

**Tips**  
If you have a sequence of ufuncs to apply, pin the data on the GPU by manual transfer

# NumbaPro CUDA Libraries
- Access to CUDA libraries
- Work seamless with Numpy
    - auto memory transfer
    - managed memory
- cuBLAS: CUDA version of BLAS
- cuSparse: CUDA sparse matrix support
- cuFFT: FFT on CUDA
- cuRNAD: random number generation on CUDA

# An Example with CUDA lib
- Convolutionon GPU

### Import cuFFT

In [77]:
from skcuda import cufft

### Misc. Imports

In [59]:
from scipy.signal import fftconvolve 
from scipy import misc, ndimage
from matplotlib import pyplot as plt
from timeit import default_timer as timer

### Build elementwise complex array multiplication CUDA function

In [60]:
@numba.vectorize(['complex64(complex64, complex64)'], target = "cuda")
def vmult(a, b):
    return a * b

### Prepare image and filter

In [61]:
image = misc.face(gray = True).astype(np.float32)

In [64]:
laplacian_pts = '''
-4 -1 0 -1 -4
-1 2 3 2 -1
0 3 4 3 0
-1 2 3 2 -1
-4 -1 0 -1 -4
'''.split()
laplacian = np.array(laplacian_pts, dtype = np.float32).reshape(5, 5)
laplacian

array([[-4., -1.,  0., -1., -4.],
       [-1.,  2.,  3.,  2., -1.],
       [ 0.,  3.,  4.,  3.,  0.],
       [-1.,  2.,  3.,  2., -1.],
       [-4., -1.,  0., -1., -4.]], dtype=float32)

In [65]:
response = np.zeros_like(image)
response[:5, :5] = laplacian

In [67]:
print("Image size: %s" % (image.shape, ))

Image size: (768, 1024)


### Convolution on the CPU

In [68]:
ts = timer() # start Timer
cvimage_cpu = fftconvolve(image, laplacian, mode = "same")
te = timer() # stop Timer
print('CPU: %.2fs' % (te - ts))

CPU: 0.06s


### Convolution on the GPU

In [70]:
image_complex = image.astype(np.complex64)
response_complex = response.astype(np.complex64)

In [71]:
ts = timer() # start Timer

d_image_complex = numba.cuda.to_device(image_complex)
d_response_complex = numba.cuda.to_device(response_complex)

# Forward FFT
cufft.fft_inplace(d_image_complex)
cufft.fft_inplace(d_response_complex)

# Multiply the image with the filter
vmult(d_image_complex, d_response_complex, out = d_image_complex)

# Inverse FFT
cufft.ifft_inplace(d_image_complex)
cvimage_gpu = d_image_complex.copy_to_host().real / np.prod(image.shape)

te = timer() # stop Timer
print('GPU: %.2fs' % (te - ts))

AttributeError: module 'skcuda.cufft' has no attribute 'fft_inplace'

# Low-level Approach: @numba.cuda.jit
- Numba can generate CUDA functions with the @jit decorator
- decorated function follows CUDA execution model

## CUDA Execution Model
- Kernel functions (created when you decorate a function)
    - visible to the host CPU
    - connot return any value
        - use output argument
    - associates to a grid
- Grid
    - a group of blocks
    - 1D, 2D, 3D
- Blocks
    - a group of threads
    - 1D, 2D, 3D
- Every thread executes the same kernel
    - thread can use the grid, block, thread coordinate system to determines its ID    

![](http://cuda.ce.rit.edu/cuda_overview/clip_image004.jpg)

# Compiling a CUDA Kernel

In [7]:
import numpy as np
import math

import numba
from numba import cuda

In [3]:
print(cuda.threadIdx.x)
print(cuda.blockIdx.x)
print(cuda.blockDim.x)   # number of threads per block 

<macro tid.x -> () -> int32>
<macro ctaid.x -> () -> int32>
<macro ntid.x -> () -> int32>


In [4]:
@cuda.jit("void(float32[:], float32[:], float32[:])")
def vadd(arr_a, arr_b, arr_out):
    # thread coordinate system
    tx = cuda.threadIdx.x
    bx = cuda.blockIdx.x
    bw = cuda.blockDim.x
    
    # flatten the coordinate system into indices
    i = tx + bx + bw
    
    # why are we checking (see later in notebooks)
    if i >= arr_out.size:
        return
    arr_out[i] = arr_a[i] + arr_b[i]

**Note: why does the kernel function cannot return any value?**
execution -> map to the cuda execution model    
the hardware of CUDA pose an restrictions  
one restriction is the output must be passed as an argument  

when you call up kernel, after the launch, the kernel is actually not finished and therefore you will never get the return value

<s>synchronizationator</s> read the output from the global memory of GPU

## Code explained

**Define a CUDA kernel with three 1D float32 arrays as args**
```
@cuda.jit("void(float32[:], float32[:], float32[:])")
def vadd(arr_a, arr_b, arr_out):
```

**Map thread, block coordinate to global position**
```
    tx = cuda.threadIdx.x
    bx = cuda.blockIdx.x
    bw = cuda.blockDim.x
    i = tx + bx + bw
```
or simplified to:
```
    i = cuda.grid(1)
```

**Ensure global position is within array size**
```
    if i >= arr_out.size:
        return
```

**The actual work**
```
    arr_out[i] = arr_a[i] + arr_b[i]
```

## Launch kernel

**Prepare data**

In [8]:
n = 100
a = np.arange(n, dtype = np.float32)
b = np.arange(n, dtype = np.float32)
c = np.empty_like(a) # Must prepare the output array to hold the result

**Calculate thread, block count**
- thread count is set to warp size of teh GPU
    - Warp size is similar to SIMD vector width on the CPU
    - performance tips: set thread count to multiple of warp size
- block count is ceil(n / thread_ct)

**Note:** This will lauch more threads than there are elements in the array

In [10]:
my_gpu = numba.cuda.get_current_device()
thread_ct = my_gpu.WARP_SIZE
block_ct = int(math.ceil(float(n) / thread_ct))

print("Threads per block:", thread_ct)
print("Block per grid:", block_ct)

Threads per block: 32
Block per grid: 4


In [11]:
a + b

array([  0.,   2.,   4.,   6.,   8.,  10.,  12.,  14.,  16.,  18.,  20.,
        22.,  24.,  26.,  28.,  30.,  32.,  34.,  36.,  38.,  40.,  42.,
        44.,  46.,  48.,  50.,  52.,  54.,  56.,  58.,  60.,  62.,  64.,
        66.,  68.,  70.,  72.,  74.,  76.,  78.,  80.,  82.,  84.,  86.,
        88.,  90.,  92.,  94.,  96.,  98., 100., 102., 104., 106., 108.,
       110., 112., 114., 116., 118., 120., 122., 124., 126., 128., 130.,
       132., 134., 136., 138., 140., 142., 144., 146., 148., 150., 152.,
       154., 156., 158., 160., 162., 164., 166., 168., 170., 172., 174.,
       176., 178., 180., 182., 184., 186., 188., 190., 192., 194., 196.,
       198.], dtype=float32)

In [12]:
# Last argument is the output array in this case
vadd[block_ct, thread_ct](a, b, c)
print(c)

[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  31.  64.  66.  68.  70.  72.  74.  76.  78.  80.  82.
  84.  86.  88.  90.  92.  94.  96.  98. 100. 102. 104. 106. 108. 110.
 112. 114. 116. 118. 120. 122. 124. 126. 128. 130. 132.  67.  68.  69.
  70.  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.
  84.  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.
  98.  99.]


The above results are not correct, however, it works properly if I change the code into the following:

In [13]:
@cuda.jit("void(float32[:], float32[:], float32[:])")
def vadd(arr_a, arr_b, arr_out):
    # thread coordinate system
    #tx = cuda.threadIdx.x
    #bx = cuda.blockIdx.x
    #bw = cuda.blockDim.x
    
    # flatten the coordinate system into indices
    #i = tx + bx + bw
    i = cuda.grid(1)
    
    # why are we checking (see later in notebooks)
    #if i >= arr_out.size:
    #    return
    arr_out[i] = arr_a[i] + arr_b[i]

In [14]:
n = 100
a = np.arange(n, dtype = np.float32)
b = np.arange(n, dtype = np.float32)
c = np.empty_like(a) # Must prepare the output array to hold the result

# Last argument is the output array in this case
vadd[block_ct, thread_ct](a, b, c)
print(c)

[  0.   2.   4.   6.   8.  10.  12.  14.  16.  18.  20.  22.  24.  26.
  28.  30.  32.  34.  36.  38.  40.  42.  44.  46.  48.  50.  52.  54.
  56.  58.  60.  62.  64.  66.  68.  70.  72.  74.  76.  78.  80.  82.
  84.  86.  88.  90.  92.  94.  96.  98. 100. 102. 104. 106. 108. 110.
 112. 114. 116. 118. 120. 122. 124. 126. 128. 130. 132. 134. 136. 138.
 140. 142. 144. 146. 148. 150. 152. 154. 156. 158. 160. 162. 164. 166.
 168. 170. 172. 174. 176. 178. 180. 182. 184. 186. 188. 190. 192. 194.
 196. 198.]


## Example: Matrix-matrix multiplication
- Show manual caching with shared memory
- Not the best matrix matrix multiplication implementation

**Prepare constants**

In [15]:
from numba import float32

bpg = 150
tpb = 32
n = bpg + tpb
shared_mem_size = (tpb, tpb)
griddim = bpg, bpg
blackdim = tpb, tpb

In [16]:
print(n)

182


In [17]:
@cuda.jit
def increment_a_2D_array(an_array):
    x, y = cuda.grid(2)
    if x < an_array.shape[0] and y < an_array.shape[1]:
        an_array[x, y] += 1

In [21]:
@cuda.jit("void(float32[:], float32[:], float32[:])")
def naive_matrix_mult(A, B, C):
    x, y = cuda.grid(2)
    #if (x >= n) or (y >= n):
    #    return
    if x < C.shape[0] and y < C.shape[1]:
        C[x, y] += 1
    #C[y, x] = 0.0
    #for i in range(n):
    #    C[y, x] += A[y, i] * B[i, x]

TypingError: Failed at nopython (nopython frontend)
Internal error at <numba.typeinfer.StaticGetItemConstraint object at 0x7f98341d03c8>:
--%<-----------------------------------------------------------------
Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/numba/errors.py", line 243, in new_error_context
    yield
  File "/usr/lib/python3/dist-packages/numba/typeinfer.py", line 321, in __call__
    index=self.index)
  File "/usr/lib/python3/dist-packages/numba/typing/context.py", line 249, in resolve_static_getitem
    return self.resolve_function_type("static_getitem", args, kws)
  File "/usr/lib/python3/dist-packages/numba/typing/context.py", line 189, in resolve_function_type
    res = defn.apply(args, kws)
  File "/usr/lib/python3/dist-packages/numba/typing/templates.py", line 193, in apply
    sig = generic(args, kws)
  File "/usr/lib/python3/dist-packages/numba/typing/builtins.py", line 526, in generic
    return tup.types[idx]
IndexError: tuple index out